In [65]:
import cv2
import numpy as np
theImage=cv2.imread("Lenna.jpg")
#先把彩色圖片轉成黑白的
theImageGray=cv2.cvtColor(theImage,cv2.COLOR_BGR2GRAY)
#秀出黑白圖片
cv2.imshow("show photo", theImageGray)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [66]:
DitherArray=np.array([[0.513,0.272,0.724,0.483,0.543,0.302,0.694,0.453],
 [0.151,0.755,0.091,0.966,0.181,0.758,0.121,0.936],
 [0.634,0.392,0.574,0.332,0.664,0.423,0.604,0.362],
 [0.060,0.875,0.211,0.815,0.030,0.906,0.241,0.845],
 [0.543,0.302,0.694,0.453,0.513,0.272,0.724,0.483],
 [0.181,0.758,0.121,0.936,0.151,0.755,0.091,0.936],
 [0.664,0.423,0.604,0.362,0.634,0.392,0.574,0.332],
 [0.030,0.906,0.241,0.845,0.060,0.875,0.211,0.815]])*255

In [67]:
#Ordered Dithering
theDitheredImage=theImageGray
height,width=theDitheredImage.shape
for i in range(0,height,8):
    for j in range(0,width,8):
        theDitheredImage[i:i+8,j:j+8]=(DitherArray>theDitheredImage[i:i+8,j:j+8])*255


#秀出圖片
cv2.imshow("show photo", theDitheredImage)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#Error Diffusion
errorArray=np.array([[0.0,    0.0,     0.0,     0.19040, 0.095230],
 [0.04762,0.095230,0.19040, 0.095230,0.04762 ],
 [0.02381,0.047620,0.095230,0.047620,0.02381]])

theErrorDiffusedImage=theImageGray
height,width=theErrorDiffusedImage.shape

tempArray=np.zeros((height+4,width+4))
tempArray[2:height+2,2:width+2]=theErrorDiffusedImage
for i in range(2,height+2):
    for j in range(2,width+2):
        #TODO:把內容補完
        old = tempArray[i, j]
        new = (old // (255//2))*(255 //(2-1))
        tempArray[i,j] = new
        E = old -new
        tempArray[i:i+3,j-2:j+3] = tempArray[i:i+3,j-2:j+3] +E*errorArray

theErrorDiffusedImage=tempArray[2:height+2,2:width+2]
#秀出圖片
cv2.imshow("show photo", theErrorDiffusedImage)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()
